In [ ]:
#Additional validation
#includes positive label adjustment


In [16]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# -----------------------------
# 1. Config
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#new_data_root = "/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/"  # <-- change this
new_data_root = "/90daydata/nematode_ml/BLD/nematode_project/additional test/images/"  # <-- change this
batch_size = 16
img_size = 384

#weights_path = "/90daydata/nematode_ml/BLD/nematode_project/outputs/20 epoch wait/efficientnet_final_model.pth"
#weights_path = "/90daydata/nematode_ml/BLD/nematode_project/outputs/10 epoch wait/efficientnet_final_model.pth"

#weights_path = "/90daydata/nematode_ml/BLD/nematode_project/outputs/5 epoch wait/efficientnet_final_model.pth"
#weights_path = "/90daydata/nematode_ml/BLD/nematode_project/outputs/5 epoch wait/efficientnet_aphid_final_model.pth"
weights_path = "/90daydata/nematode_ml/BLD/nematode_project/outputs/5 epoch wait/efficientnet_aphids_final_model.pth"

#weights_path = "/90daydata/nematode_ml/BLD/nematode_project/outputs/EfficientNet/efficientnet_final_model.pth"
#weights_path = "/90daydata/nematode_ml/BLD/nematode_project/final_model.pth"

In [17]:
# -----------------------------
# 2. Transforms (same as val set)
# -----------------------------
MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)
eval_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD),
])

# -----------------------------
# 3. Dataset & Loader
# -----------------------------
dataset = torchvision.datasets.ImageFolder(new_data_root, transform=eval_tf)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [18]:
# -----------------------------
# 4. Model definition & load weights
# -----------------------------+
model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 1)  # binary classification

model.load_state_dict(torch.load(weights_path, map_location=device))
model.to(device)
model.eval()


/local/scratch/benjamin.waldo/18989625/ipykernel_424820/2690978171.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights_path, map_loc

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [19]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# -----------------------------
# 5. Evaluation loop (auto label alignment)
# -----------------------------
# Prepares lists to collect labels, predictions, probabilities, and paths
all_labels = []
all_preds = []
all_probs = []
all_paths = []   # NEW: store image paths

# Detect positive label value from dataset if possible
class_to_idx = getattr(getattr(loader, 'dataset', None), 'class_to_idx', None)
if class_to_idx is not None and "BLD" in class_to_idx:
    pos_label_val = class_to_idx["BLD"]
else:
    pos_label_val = 1  # fallback

with torch.no_grad():
    # If your dataset returns (image, label, path), unpack accordingly
    for batch in loader:
        if len(batch) == 3:
            inputs, labels, paths = batch
        else:
            inputs, labels = batch
            # fallback: try to get paths from dataset.samples
            paths = [loader.dataset.samples[i][0] for i in range(len(labels))]

        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        probs = torch.sigmoid(outputs).squeeze()  # convert logits to probabilities

        preds = (probs > 0.5).long()  # Thresholds at 0.5 to get binary predictions

        # Collects labels, predictions, probabilities, and paths
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        all_paths.extend(paths)

# -----------------------------
# 6. Metrics
# -----------------------------
# Converts lists to NumPy arrays
y_true = np.array(all_labels)
y_pred = np.array(all_preds)
y_score = np.array(all_probs)

# Flip labels if dataset encodes BLD as 0
if pos_label_val != 1:
    y_true = 1 - y_true

acc  = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec  = recall_score(y_true, y_pred, zero_division=0)
f1   = f1_score(y_true, y_pred, zero_division=0)
try:
    auroc = roc_auc_score(y_true, y_score)
except ValueError:
    auroc = float("nan")

print(f"Unseen dataset results:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 score:  {f1:.4f}")
print(f"AUROC:     {auroc:.4f}")

# -----------------------------
# 7. Misclassified Images
# -----------------------------
misclassified_idx = np.where(y_true != y_pred)[0]

# Step 1: Deduplicate Misclassified Paths
misclassified = {}
for i in misclassified_idx:
    path = all_paths[i]
    misclassified[path] = (y_true[i], y_pred[i])

print("\nUnique misclassified images:")
for path, (true, pred) in misclassified.items():
    print(f"Path: {path}, True: {true}, Pred: {pred}")


Unseen dataset results:
Accuracy:  0.9655
Precision: 0.9184
Recall:    1.0000
F1 score:  0.9574
AUROC:     0.9987

Unique misclassified images:
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51293908837_cc045f7490_o.jpg, True: 0, Pred: 1
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51293908882_f7ddc1b52c_o.jpg, True: 0, Pred: 1
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51295376964_e287ede9dd_o.jpg, True: 0, Pred: 1
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51293909007_9b6eb53479_o.jpg, True: 0, Pred: 1


In [15]:
# -----------------------------
# Diagnostic: Check class encoding
# -----------------------------
print("\nClass-to-Index mapping:", class_to_idx)

# Show a few sample paths and labels
for i in range(5):
    path, label = dataset.samples[i]
    print(f"Sample {i}: Path={path}, Label={label}")



Class-to-Index mapping: {'0_No_BLD': 0, '1_BLD': 1}
Sample 0: Path=/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/0_No_BLD/IMG_7634.JPG, Label=0
Sample 1: Path=/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/0_No_BLD/IMG_7634.jpg, Label=0
Sample 2: Path=/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/0_No_BLD/IMG_7635.JPG, Label=0
Sample 3: Path=/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/0_No_BLD/IMG_7635.jpg, Label=0
Sample 4: Path=/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/0_No_BLD/IMG_7639.JPG, Label=0


In [8]:
print("Class-to-Index mapping:", dataset.class_to_idx)


Class-to-Index mapping: {'BLD': 0, 'No_BLD': 1}


In [9]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)



Confusion Matrix:
[[67  4]
 [ 0 45]]


In [10]:
# Identify false positives (No_BLD misclassified as BLD)
false_positive_idx = np.where((y_true == 0) & (y_pred == 1))[0]

print("\nFalse Positive Images (No_BLD predicted as BLD):")
for i in false_positive_idx:
    print(f"Path: {all_paths[i]}, Prob: {y_score[i]:.4f}")



False Positive Images (No_BLD predicted as BLD):
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51293908837_cc045f7490_o.jpg, Prob: 0.8862
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51293908882_f7ddc1b52c_o.jpg, Prob: 0.7112
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51295376964_e287ede9dd_o.jpg, Prob: 0.7633
Path: /90daydata/nematode_ml/BLD/nematode_project/additional test/images/BLD/51293909007_9b6eb53479_o.jpg, Prob: 0.7684


In [5]:
for path, label in dataset.samples[:10]:
    print(path, label)


/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51293908837_cc045f7490_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51293908882_f7ddc1b52c_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51293908897_290b78d149_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51293908952_e8f1f1c91e_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51293909007_9b6eb53479_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51294659401_4bdac1d46b_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51294834178_1144dfcddf_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51295371824_1e4a8d8a47_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51295372019_a3e6f610fe_o.jpg 0
/90daydata/nematode_ml/BLD/nematode_project/additional test/Aphids/BLD/51295372339